# Deploy Llama3-8b-Instruct on AWS Inferentia2 using Amazon SageMaker and Huggingface TGI container

[Text Generation Inference (TGI)](https://huggingface.co/docs/optimum-neuron/guides/neuronx_tgi), is a purpose-built solution for deploying and serving Large Language Models (LLMs) for production workloads at scale. TGI enables high-performance text generation using Tensor Parallelism and continuous batching for the most popular open LLMs, including Llama, Mistral, and more. 

The integration of TGI into Amazon SageMaker, in combination with AWS Inferentia2, presents a powerful solution and viable alternative to GPUs for building production LLM applications. The seamless integration ensures easy deployment and maintenance of models, making LLMs more accessible and scalable for a wide range of production use cases.

With the new TGI for AWS Inferentia2 on Amazon SageMaker, AWS customers can benefit from the same technologies that power highly-concurrent, low-latency LLM experiences like HuggingChat, OpenAssistant, and Serverless Endpoints for LLMs on the Hugging Face Hub.


In this notebook, you'll do the following:

1. Setup development environment
2. Retrieve the TGI Neuronx Image
3. Deploy [Llama3-8B-Instruct](https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct) to Amazon SageMaker
4. Run inference and chat with the model

## 1. Install, import the required libraries, and set some variables

In [ ]:
!pip install transformers "sagemaker>=2.206.0" --upgrade --quiet

Note: You might have to restart the kernel to use the installed packages

In [ ]:
import sagemaker
import boto3
sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it doesn't exist
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker session region: {sess.boto_region_name}")


## 2. Retrieve TGI Neuronx Image

The new Hugging Face TGI Neuronx DLCs can be used to run inference on AWS Inferentia2. You can use the get_huggingface_llm_image_uri method of the sagemaker SDK to retrieve the appropriate Hugging Face TGI Neuronx DLC URI based on your desired backend, session, region, and version. You can find all the available versions [here](https://github.com/aws/deep-learning-containers/releases?q=tgi+AND+neuronx&expanded=true).


In [ ]:
from sagemaker.huggingface import get_huggingface_llm_image_uri

# retrieve the llm image uri
llm_image = get_huggingface_llm_image_uri(
  "huggingface-neuronx",
  version="0.0.23"
)

# print ecr image uri
print(f"llm image uri: {llm_image}")


## 3. Deploy Llama3 8B Instruct on Amazon SageMaker


[Text Generation Inference (TGI)](https://github.com/huggingface/optimum-neuron/tree/main/text-generation-inference) on Inferentia2 supports popular open LLMs, including Llama, Mistral, and more. [Supported Architectures](https://huggingface.co/docs/optimum-neuron/en/package_reference/supported_models) has a list of supported models.


### Compiling LLMs for Inferentia2

To deploy models on AWS Inferentia2, we need to compile the model with a sequence length and batch size ahead of time. To make it easier for customers to utilize the full power of Inferentia2, we created a [neuron model cache](https://huggingface.co/aws-neuron/optimum-neuron-cache), which contains pre-compiled configurations for the most popular LLMs. A cached configuration is defined through a model architecture (Llama3), model size (8B), neuron version (2.18), number of inferentia cores (2), batch size (4), and sequence length (4096).

This means that when deploying models with an architecture based on Llama3 and a configuration for which Neuron compiled artifacts exist; there will be no need to re-compile your model.


In [ ]:
import json
from sagemaker.huggingface import HuggingFaceModel

# sagemaker config & model config
instance_type = "ml.inf2.8xlarge"
health_check_timeout = 3600

# Define Model and Endpoint configuration parameter
config = {
    "HF_MODEL_ID": "meta-llama/Meta-Llama-3-8B-Instruct",
    "HF_NUM_CORES": "2",
    "HF_BATCH_SIZE": "4",
    "HF_SEQUENCE_LENGTH": "4096",
    "HF_AUTO_CAST_TYPE": "fp16",
    "MAX_BATCH_SIZE": "4",
    "MAX_INPUT_LENGTH": "3686",
    "MAX_TOTAL_TOKENS": "4096",
    "HF_TOKEN": <YOUR_HF_TOKEN>,
}

# create HuggingFaceModel with the image uri
llm_model = HuggingFaceModel(
  role=role,
  image_uri=llm_image,
  env=config
)

In [ ]:
# Deploy model to an endpoint
llm = llm_model.deploy(
  initial_instance_count=1,
  instance_type=instance_type,
  container_startup_health_check_timeout=1800,
  volume_size=512
)

NOTE: Please ignore the warning above that says "Your model is not compiled. Please compile your model before using Inferentia."

### The above step takes about 10-15 minutes.

While you wait for the model to be deployed, you can read the below resources - 
- [AWS Neuron](https://awsdocs-neuron.readthedocs-hosted.com/en/latest/index.html)
- [AWS Inferentia2](https://awsdocs-neuron.readthedocs-hosted.com/en/latest/general/arch/neuron-hardware/inf2-arch.html)
- [Amazon SageMaker with HuggingFace Optimum Neuron](https://huggingface.co/docs/optimum-neuron/en/guides/sagemaker)

## Invoke the Endpoint

In [ ]:
from transformers import AutoTokenizer

# load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    "meta-llama/Meta-Llama-3-8B-Instruct",
    token=<YOUR_HF_TOKEN>
)

# Prompt to generate
messages = [
    {"role": "system", "content": "You are the AWS expert"},
    {"role": "user", "content": "Can you tell me an interesting fact about AWS?"},
]
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

# Generation arguments
parameters = {
    "do_sample": True,
    "top_p": 0.6,
    "temperature": 0.9,
    "max_new_tokens": 1024,
    "return_full_text": False,
}

res = llm.predict({"inputs": prompt, "parameters": parameters})
print(res[0]["generated_text"].strip().replace("</s>", ""))

In [ ]:
llm.delete_model()
llm.delete_endpoint()